In [1]:
import argparse
import pandas as pd
import numpy as np
import peddy ### must be in python version 3.7 for peddy to actually work


In [ ]:
ped = peddy.Ped('lynchWGS.ped.txt', 'Paternal_ID' == str) ###import peddy file

In [ ]:
def has_parents(sample):
    """Check if Peddy sample has both parents in ped file"""
    if sample.mom is not None and sample.dad is not None:
        return True
    return False

In [ ]:
df = pd.read_table('LaurelSTRsupdated.tsv', delim_whitespace = True, index_col = 0, dtype = {'sample' : str})

In [ ]:
dfnew = df[['allele1_est', 'allele2_est']]

In [ ]:
dfnewallele1sorted = dfnew.sort_values(by=['allele1_est'])

In [ ]:
newdf2 = dfnew.groupby(['allele2_est', 'allele1_est']).first()

In [3]:
newdf2

NameError: name 'newdf2' is not defined

In [14]:
mom_alleles = ((340, 360), (340, 360))

In [ ]:
compare_alleles

In [2]:
def compare_alleles(mom, dad, kid):
    results = {}
    for i in mom_alleles:
        results[i] = kid_alleles.count(i) 
    print(results)
    
    results = {}
    for i in dad_alleles:
        results[i] = kid_alleles.count(i) 
    print(results)
    

In [ ]:
allele_list = []
for x in valid_alleles(parent_alleles):
    allele_list.append(x)

In [ ]:
flat_list = list(np.concatenate(allele_list).flat)

In [ ]:
sorted(flat_list)

In [ ]:
compare_alleles(mom_alleles, dad_alleles, kid_alleles)
##could do loop
## could do list comprehension but


    

In [ ]:
sum = 0
sum += dict[key]
## for key in dictionary:

In [ ]:
momkid = (set(mom_alleles) & set(kid_alleles))

In [ ]:
allele = 100

In [ ]:
wigglecommand = 10

In [ ]:
wiggle(wigglecommand)

In [4]:
def wiggle(wigglecommand):
        if type(wigglecommand) is int:
            wiggleint = wigglecommand
            return wiggleint
        else:
            type(wigglecommand) is str
            wigglestr = int(wigglecommand)
            return wigglestr

In [5]:
def allele_check(allele1,allele2):
    if allele1 == 'NaN' and allele2 == 'NaN':
        allele1 == 'NaN'
    elif allele1 != 'NaN' and allele2 == 'NaN':
        if allele1 >= 350:
            allele2 = 350
            allele1 = 350
        else:
            allele2 = allele1
    elif allele1 == 'NaN' and allele2 != 'NaN':
        if allele2 >= 350:
            allele1 = 350
            allele2 = 350
        else:
            allele1 = allele2
    elif allele2 >= 350:
        allele2 = 350   
    elif allele1 >= 350:
        allele1 = 350
    else:
        allele1 = allele1
        allele2 = allele2
    return allele1, allele2

In [6]:
allele_check(50, 60)

(50, 60)

In [9]:
allele_range(10,150,150)

((140, 160), (140, 160))

In [8]:
def allele_range(wigglecommand, allele1, allele2):
    if allele1 and allele2 == 'NaN':
      return  
    else:
        tple1 = (allele1 - wiggle(wigglecommand), allele1 + wiggle(wigglecommand))
        tple2 = (allele2 - wiggle(wigglecommand), allele2 + wiggle(wigglecommand))
    return tple1, tple2

    

In [30]:
def get_allele_ranges(wigglecommand, allele1, allele2):
    a, b = allele_check(allele1,allele2)
    x, y =(allele_range(wigglecommand, a, b))
    return x, y
    
    

In [31]:
get_allele_ranges(10,500,'NaN')

((340, 360), (340, 360))

In [25]:
def setup_check_range(wigglecommand, tuplee1, tuplee2):
    print(x)
    print(y)

In [50]:
def check_range(wigglecommand, allele1, allele2, kidallele):
    x,y = get_allele_ranges(wigglecommand, allele1, allele2)
    a,b = x
    c,d = y
    if a <= kidallele <= b or c <= kidallele <= d:
        return True
    elif kidallele > a and b:
        return 'Too big'
    else: 
        print('Booooo')

In [51]:
check_range(10, 150,130,120)

True

In [19]:
check_range(kidallele, allele_range(wigglecommand, mom_allele1))
check_range(kidallele, allele_range(wigglecommand, mom_allele2))

NameError: name 'kidallele' is not defined

In [20]:
if check_range(kidallele, allele_range(wigglecommand, mom_allele1)) is True:
    print("Cool")
else:
    print('What is up')

NameError: name 'kidallele' is not defined

In [ ]:
mom_allele2 = 200